The objective to this is to concentrate ourselves to demand forecast with just Blastness dataset and nothing more:
- We have to clean the data provided and see patterns.
- We will compare random forests and prophet neural network.

The idea is to advance more towards HotelRank part of the demand for the moment without data engineering infra to fetch weather and events. 

In [50]:
!pip install bamboolib
!python -m bamboolib install_nbextensions

Trying to install bamboolib nbextension...
Could not install bamboolib Jupyter Notebook extension because Jupyter Notebook is not available


In [51]:

import os
import glob
import pandas as pd
filelist = glob.glob('./hoteldataset/*.csv')
hotelsbookings = []
column_mapping = {
    'Codice': 'Code',
    'Status': 'Status',
    'Canale': 'BookingChannel',
    'Arrivo': 'Arrival',
    'Partenza': 'Departure',
    'Notti': 'Nights',
    'Totale': 'Total',
    'Data acquisto': 'Purchase Date',
    'Origine': 'Origin',
    'Dispositivo': 'BoookingDevice',
    'Data Ultima Modifica/Cancellazione': 'Last Modification/Cancellation Date'
}
for idx,f in enumerate(filelist):
    df = pd.read_csv(f)
    select_columns = list(column_mapping.keys())
    current_df = df[select_columns]
    remap = current_df.rename(columns=column_mapping)
    hotel_id = ""
    if idx < 9:
        hotel_id=f'00{idx+1}'
    else:
        hotel_id=f'0{idx+1}'

    remap['HotelId'] = hotel_id
    hotelsbookings.append(remap)
hotelsbookings[9].head()


/tmp/ipykernel_702424/1596637349.py:20: DtypeWarning: Columns (23,52,55,58,59,60,81,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
/tmp/ipykernel_702424/1596637349.py:20: DtypeWarning: Columns (23,53,62,68,70,81) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
/tmp/ipykernel_702424/1596637349.py:20: DtypeWarning: Columns (23,49,55,59,81,87,90) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
/tmp/ipykernel_702424/1596637349.py:20: DtypeWarning: Columns (23,52,53,55,58,59,60,62,68,70,81,89,90,91) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


,Code,Status,BookingChannel,Arrival,Departure,Nights,Total,Purchase Date,Origin,BoookingDevice,Last Modification/Cancellation Date,HotelId
0,GN200KZEYQI1,Confermata,Sito,30/09/2006,01/10/2006,1,"451,00",01/09/2006,NaN,Website,01/09/2006 14:11:10,010
1,YR200KTNTGX2,Cancellata,Sito,12/09/2006,13/09/2006,0,"0,00",03/09/2006,NaN,Website,03/09/2006 20:33:41,010
2,CD200SIBGGQ5,Confermata,Sito,15/10/2006,17/10/2006,2,"583,00",04/09/2006,NaN,Website,04/09/2006 18:35:23,010
3,KL200QSBLYE4,Confermata,Sito,15/10/2006,17/10/2006,2,"583,00",04/09/2006,NaN,Website,04/09/2006 18:35:23,010
4,SW200VWGTGG3,Confermata,Sito,15/10/2006,17/10/2006,2,"583,00",04/09/2006,NaN,Website,04/09/2006 18:35:23,010


Now we merge all hotels in a single dataframe and save to disk. We note that we need to divide cancelled and confirmed booking. Later since we want do forecast on the confirmed.
- Also we need to categorize the origin
- remove the booking device.

First we merge.

In [52]:
# Merge DataFrames
merged_df = pd.concat(hotelsbookings, ignore_index=True)
# Display the merged DataFrame
print("Merged DataFrame:")
print(merged_df)

Merged DataFrame:
                         Code      Status BookingChannel     Arrival  \
0        178054733/1050553517  Confermata    Booking.com  04/01/2017   
1       1854547257/1135482031  Confermata    Booking.com  10/12/2016   
2       2081545918/1135999314  Cancellata    Booking.com  18/12/2016   
3       1683692499/1137141862  Cancellata    Booking.com  07/12/2016   
4       1683692499/1137141870  Cancellata    Booking.com  07/12/2016   
...                       ...         ...            ...         ...   
151852           XA200DO52912  Confermata           Sito  25/07/2023   
151853           EZ200NJ52913  Confermata           Sito  25/07/2023   
151854           GG200OT52914  Confermata           Sito  26/05/2023   
151855           MG200OY52915  Confermata           Sito  28/06/2023   
151856           QB200WI52916  Confermata           Sito  28/06/2023   

         Departure  Nights    Total Purchase Date Origin BoookingDevice  \
0       09/01/2017       5  2288,50    01/

Ok, we've seen a lot of not known or bad data. Just clean it. We want order by date, in descending mode and take only the last 2 years

In [53]:
merged_df.head()

,Code,Status,BookingChannel,Arrival,Departure,Nights,Total,Purchase Date,Origin,BoookingDevice,Last Modification/Cancellation Date,HotelId
0,178054733/1050553517,Confermata,Booking.com,04/01/2017,09/01/2017,5,"2288,50",01/08/2016,NaN,NaN,20/12/2016 14:27:05,001
1,1854547257/1135482031,Confermata,Booking.com,10/12/2016,12/12/2016,2,"424,80",19/10/2016,NaN,NaN,19/10/2016 22:29:19,001
2,2081545918/1135999314,Cancellata,Booking.com,18/12/2016,20/12/2016,0,"0,00",20/10/2016,NaN,NaN,20/10/2016 13:25:40,001
3,1683692499/1137141862,Cancellata,Booking.com,07/12/2016,11/12/2016,0,"0,00",21/10/2016,NaN,NaN,21/10/2016 15:48:07,001
4,1683692499/1137141870,Cancellata,Booking.com,07/12/2016,11/12/2016,0,"0,00",21/10/2016,NaN,NaN,21/10/2016 15:48:07,001


In [56]:
merged_df['Arrival'] = pd.to_datetime(merged_df['Arrival'])
# Sorting by 'Arrival' column in descending order
sorted_bookings_df = merged_df.sort_values(by='Arrival', ascending=False)
# Filtering for HotelId '001'
filtered_df = sorted_bookings_df.loc[sorted_bookings_df['HotelId'] == '010']
filtered_df.head()

,Code,Status,BookingChannel,Arrival,Departure,Nights,Total,Purchase Date,Origin,BoookingDevice,Last Modification/Cancellation Date,HotelId
151689,EQ200XD52749,Confermata,Sito,2023-12-07,15/07/2023,3,"1663,20",27/11/2022,NaN,Website,27/11/2022 07:27:09,010
151017,8357485/1,Confermata,HotelBeds,2023-12-06,15/06/2023,3,"1449,00",22/10/2022,NaN,NaN,22/10/2022 00:11:12,010
149939,8233730/1,Cancellata,HotelBeds,2023-12-06,15/06/2023,0,"0,00",14/09/2022,NaN,NaN,14/09/2022 16:42:34,010
150810,20022166570845744,Confermata,Galileo,2023-12-06,15/06/2023,3,"1740,00",14/10/2022,NaN,NaN,14/10/2022 02:47:37,010
150811,200745166570853524,Confermata,Galileo,2023-12-06,15/06/2023,3,"1350,00",14/10/2022,NaN,NaN,14/10/2022 02:48:55,010
